## Cleaning

In [ ]:
import pandas as pd
from tqdm import tqdm

filename = 'twitter_cs'

twcs: pd.DataFrame = pd.read_csv(f'data/{filename}.csv', nrows=100_000)
twcs = twcs.sort_values('created_at')
twcs.head()

In [ ]:
twcs["text"] = (
    twcs["text"]
    .str.replace(r"^\s*@[^ ]*", "", regex=True)
    .str.replace(r"https?:\/\/[^\s\\n]+", "", regex=True)
    .str.replace(r"\n+", ' ', regex=True)
    .str.strip()
)
twcs = twcs.rename(columns={'inbound': 'is_customer'})

### Make threads

In [ ]:
def find_root(tweet_id, df):
    parent_id = tweet_id

    while True:
        potential_parent = df[df['in_response_to_tweet_id'] == parent_id]['tweet_id']

        if len(potential_parent) == 0:
            return parent_id
        
        parent_id = potential_parent.values[0]

tqdm.pandas(desc="Making threads...")
twcs['thread_id'] = twcs['tweet_id'].progress_apply(lambda x: find_root(x, twcs))

### Aggregate to chats

In [ ]:
def group_conversations(df):
    altnerating_messages = []
    prev_is_customer = None

    for _, row in df.iterrows():
        is_customer = row['is_customer']

        if prev_is_customer is None and not is_customer: # Always start with a customer message
            continue
        elif prev_is_customer == is_customer:
            altnerating_messages[-1] += ' ' + row['text']
        else:
            altnerating_messages.append(row['text'])
            prev_is_customer = is_customer

    return altnerating_messages

chats_df = twcs.copy()
tqdm.pandas(desc="Grouping conversations...")
chats_df['chat'] = chats_df.progress_apply(lambda x: group_conversations(twcs[twcs['thread_id'] == x['thread_id']]), axis=1)
chats_df = chats_df.drop_duplicates('thread_id')[['chat']]
chats_df['n_messages'] = chats_df['chat'].apply(lambda x: len(x))

### Filter

In [ ]:
proper_length = chats_df['n_messages'] >= 4
non_dm = chats_df['chat'].apply(lambda c: all([' dm' not in m.lower() for m in c]))

chats_df = chats_df[proper_length & non_dm]
print(f"Found {len(chats_df)} fitting chats")

## Augmentation

### Embed random question

In [ ]:
import numpy as np

def random_customer_idx(x):
    even = [i for i in range(0, x - 2, 2)] # Don't pick the last message
    return np.random.choice(even)

chats_df['aug_idx'] = chats_df['n_messages'].apply(lambda x: random_customer_idx(x))
chats_df['aug_text'] = chats_df.apply(lambda x: x['chat'][x['aug_idx']], axis=1)

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    # "mixedbread-ai/mxbai-embed-large-v1"
    "TaylorAI/bge-micro-v2"
)

to_embed = chats_df['aug_text'].tolist()
embeddings = model.encode(to_embed, show_progress_bar=True)
chats_df['aug_embedding'] = embeddings.tolist()

### Cluster questions

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

ks = [i for i in range(1, 50, 1)]
wcss = []

for k in ks:
    kmeans = KMeans(n_clusters=k, random_state=0).fit(embeddings)
    wcss.append(kmeans.inertia_)


plt.plot(ks, wcss)

plt.title('Elbow Method For Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('WCSS (Inertia)')

plt.show()

In [ ]:
k = 20

kmeans = KMeans(n_clusters=k, random_state=0).fit(embeddings)
chats_df['cluster'] = kmeans.labels_

### Creating augmented data

In [ ]:
def substitue_sim_answer(row, df):
    same_cluster  = df[df['cluster'] == row['cluster']]

    sub = same_cluster.loc[np.random.choice(same_cluster.index)]
    sub_chat, sub_aug_idx= sub['chat'], sub['aug_idx']
    sub_answer = sub_chat[sub_aug_idx + 1]

    orig_chat, orig_aug_idx = row['chat'], row['aug_idx']
    aug_chat = orig_chat[:orig_aug_idx + 1] + [sub_answer] + orig_chat[orig_aug_idx + 2:]

    return aug_chat



chats_df['aug_chat'] = chats_df.progress_apply(lambda x: substitue_sim_answer(x, chats_df), axis=1)

In [ ]:
chats = np.array(chats_df[['chat', 'aug_chat']].values)

to_shuffle = np.random.rand(len(chats)) > 0.5

labels = np.where(to_shuffle, -1, 1)

to_shuffle = np.column_stack((to_shuffle, to_shuffle))
shuffled_chats = np.where(to_shuffle, chats[:, ::-1], chats)

## Construcing graph

### Nodes

In [ ]:
import torch

torch.save(shuffled_chats, f"data/{filename}_nodes.pt")

### Labels

In [ ]:
torch.save(labels, f"data/{filename}_labels.pt")

### Edges

In [ ]:
from torch import Tensor

def create_chat_graph(chat):
    human_idxs = [i for i in range(0, len(chat), 2)]

    chat_edges = []
    chat_edges_idxs = []
    for ui in range(len(chat)):
        for uj in range(len(chat)):
            if ui == uj:
                edge_type = [True, False, False, False]
            else:
                edge_type = [
                    False,
                    ui > uj,
                    ui in human_idxs,
                    uj in human_idxs,
                ]

            edge_type = sum(2**i for i, v in enumerate(reversed(edge_type)) if v)

            chat_edges_idxs.append((ui, uj))
            chat_edges.append(edge_type)
    
    return chat_edges, chat_edges_idxs


edges = []
edge_idxs = []
for c1, c2 in shuffled_chats:
   c1_edges, c1_edge_idxs = create_chat_graph(c1)
   c2_edges, c2_edge_idxs = create_chat_graph(c2)

   edges.append(Tensor([c1_edges, c2_edges]).to(torch.int8))
   edge_idxs.append(Tensor([c1_edge_idxs, c2_edge_idxs]).T.long())

torch.save(edges, f"data/{filename}_edges.pt")
torch.save(edge_idxs, f"data/{filename}_edge_idxs.pt")